## Setup
Import and install libraries

In [131]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install --upgrade pymupdf

from google import genai
from google.genai import types




In [132]:
# API keys
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

#for m in client.models.list():
#    if "embedContent" in m.supported_actions:
#        print(m.name)

In [133]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Upload of Arxiv papers
First import the arxiv dataset and then perform vector embedding of all the documents. After the vector embedding, it is saved in a chromadb vector database. The arxiv dataset import is shown below.

In [134]:
import json
amount_papers = 100
papers = []
with open('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', 'r') as file:
    for i, line in enumerate(file):
        if i >= amount_papers:
            break
        papers.append(json.loads(line))

# Now data is a list of dictionaries
print("Headers:", list(papers[0].keys()))

Headers: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed']


Only the title and the abstract of each paper will be embedded. The code below implements this preprocessing of the papers.

In [135]:
def remove_newlines(obj):
    if isinstance(obj, str):
        return obj.replace('\n', ' ')
        
preprocessed_papers = []
for paper in papers:
    preprocessed_papers.append("PAPER TITLE: " + remove_newlines(paper["title"]) + "\nPAPER CONTENT: "+ remove_newlines(paper["abstract"]))
print(preprocessed_papers[0])

PAPER TITLE: Calculation of prompt diphoton production cross sections at Tevatron and   LHC energies
PAPER CONTENT:   A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, s

Now the preprocessed papers are transformed into vector embeddings.

In [138]:
def batch(iterable, n=100):
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

# Example usage:
papers_embedded = []  # Your list of inputs to embed
papers_batches = list(batch(preprocessed_papers, 100))

for batch in papers_batches:
    batch_embedded = client.models.embed_content(
        model='models/text-embedding-004',
        contents=batch,
        config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY'))
    list_batch_embedded = [e.values for e in batch_embedded.embeddings]
    papers_embedded+=list_batch_embedded

print("SUCCESSFULLY EMBEDDED "+ str(len(papers_embedded)) + " PAPERS")

SUCCESSFULLY EMBEDDED 100 PAPERS


Once the vector embeddings of the papers are computed, these are stored into the chromadb database.

In [139]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
def batch(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]


# Start ChromaDB client
chromadb_client = chromadb.Client()

# Create or get a collection
collection = chromadb_client.get_or_create_collection(name="papers")

# Add the documents + embeddings to Chroma
emb_batches = list(batch(papers_embedded, 41000))
papers_batches = list(batch(preprocessed_papers, 41000))
for i in range(len(emb_batches)):
    ids_batch = [f"doc_{j + i * 41000}" for j in range(len(emb_batches[i]))]
    collection.add(
        documents=papers_batches[i],
        embeddings=emb_batches[i],
        ids=ids_batch,
    )
print("SUCCESSFULLY UPLOADED "+ str(len(papers_embedded)) + " PAPERS")

SUCCESSFULLY UPLOADED 100 PAPERS


## Vector database search example

In [140]:
query_input = "Statistical modeling of experimental physical laws is based on the probability density function of measured variables. It is expressed by experimental data via a kernel estimator. The kernel is determined objectively by the scattering of data during calibration of experimental setup. A physical law, which relates measured variables, is optimally extracted from experimental data by the conditional average estimator. It is derived directly from the kernel estimator and corresponds to a general nonparametric regression. T"
#query_input = pdf_text

query_embedding = client.models.embed_content(
        model='models/text-embedding-004',
        contents=query_input,
        config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY'))

In [146]:
results = collection.query(
    query_embeddings=[query_embedding.embeddings[0].values],
    n_results=5  # Number of similar docs to return
)

for doc, doc_id in zip(results["documents"][0], results["ids"][0]):
    print(f"ID: {doc_id}")
    print(f"{doc}\n")

ID: doc_88
PAPER TITLE: A general approach to statistical modeling of physical laws:   nonparametric regression
PAPER CONTENT:   Statistical modeling of experimental physical laws is based on the probability density function of measured variables. It is expressed by experimental data via a kernel estimator. The kernel is determined objectively by the scattering of data during calibration of experimental setup. A physical law, which relates measured variables, is optimally extracted from experimental data by the conditional average estimator. It is derived directly from the kernel estimator and corresponds to a general nonparametric regression. The proposed method is demonstrated by the modeling of a return map of noisy chaotic data. In this example, the nonparametric regression is used to predict a future value of chaotic time series from the present one. The mean predictor error is used in the definition of predictor quality, while the redundancy is expressed by the mean square distan

## GENAI AGENT
Now an AI agent is created where an LLM interacts with the user input and searches through the papers database to answer the scientific-paper related question. 

### Orchestration functions
The orchestrator is composed of the following functions:

- create_embedding(text): For a given text generates the corresponding vector embedding.
- search_embedded_documents(query_embedding, n): For a given vector, searches nearby vectors in the vector embeddings database.
- retrieve_documents(doc_id):

In [147]:
from google.genai import types

# === Tools ===
def create_embedding(text)-> list:
    print(f' - CALL: create_embedding({text[:20]})')
    vector_embedding = client.models.embed_content(
        model='models/text-embedding-004',
        contents=text,
        config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY')
    )
    return vector_embedding.embeddings[0].values

def search_embedded_documents(query_embedding:list[float], n:int)->list[str]:
    print(f' - CALL: search_embedded_documents(n = {n})')
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    return results

def retrieve_documents(doc_ids:list[int])-> list[dict]:
    print(f' - CALL: retrieve_documents(IDS = {doc_ids})')
    papers_retrieved = []
    for doc_id in doc_ids:
        papers_retrieved.append(papers[doc_id])
        
    return papers_retrieved


# === Tool declarations ===
create_embedding_tool = {
    "name" : "create_embedding",
    "description" : "For a given text, generate the corresponding vector embedding.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "text": {
                "type": "STRING",
                "description": "The input text to embed."
            }
        },
        "required": ["text"]
    }
}

search_embedded_documents_tool = {
    "name" : "search_embedded_documents",
    "description" : "Search for similar documents using a query embedding.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "query_embedding": {
                "type": "ARRAY",
                "items": {
                    "type": "NUMBER"  # O "INTEGER" si tus vectores son int (normalmente son floats)
                },
                "description": "The vector embedding of the input query."
            },
            "n": {
                "type": "INTEGER",
                "description": "Number of top similar documents to retrieve."
            }
        },
        "required": ["query_embedding", "n"]
    }
}

retrieve_documents_tool = {
    "name" : "retrieve_documents",
    "description" : "Retrieve detailed information about a document using its ID.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "doc_id": {
                "type": "INTEGER",
                "description": "The ID of the paper/document."
            }
        },
        "required": ["doc_id"]
    }
}





In [148]:
#orchestration_tools  = types.Tool(function_declarations=[create_embedding_tool, search_embedded_documents_tool])

orchestration_tools  = types.Tool(function_declarations=[create_embedding_tool, search_embedded_documents_tool])


instruction = """You are a helpful chatbot that can interact with a database of vector embeddings
of scientific papers and a database with the papers extended information. You will take the users questions and documents andgenerate

Use the following tools:
    - create_embedding(text) to convert text into vector embeddings 
    - search_embedded_documents(query_embedding, n) to obtain n papers that are similar to the embedded query 
from the database of vector embeddings.

"""





In [149]:
import pymupdf

def extract_text_from_pdf(path):
    text = ""
    with pymupdf.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text


pdf_text = extract_text_from_pdf("/kaggle/input/unc-paper/2409.10655v2.pdf")

In [150]:

user_document = pdf_text[:1000]
user_message = "Find me related papers."

instruction = """You are a helpful chatbot that processes inputs from users and generates an output text based on the user requirements. This output text is going to be used to be used to do vector search in a database of vector embedded papers."""

contents = [
    types.Content(
        role="user", parts=[types.Part(text=user_message),types.Part(text=pdf_text)]
    )
]

response = client.models.generate_content(
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[orchestration_tools]
    ),
    contents = contents
)


In [151]:
embedding = create_embedding(response.text)
search_output = search_embedded_documents(embedding, 5)
extended_info = retrieve_documents([89,90,91,92])

instruction = """ You are a helpful chatbot that researched on a papers databased and obtained related papers. Answer the first user query with the found papers"""
instruction = """You are a helpful chatbot that uses retrieval augmented generation to answer questions regarding scientific papers. The user provided a QUESTION and an INPUT_DOCUMENT. 
                The information retreived from the database is composed of: 
                - EMBED_DATA: Shows the documents obtained after a vector search from the embedding of QUESTION and INPUT_DOCUMENT and the database.
                - EXTENDED_PAPER_INFO: Shows more information of the papers from EMBED_DATA.

                Answer the user QUESTION using the EMBED_DATA and EXTENDED_PAPER_INFO. IF the EMBED_DATA has the paper from INPUT_DOCUMENT, skip the paper in the answer.
                
                """

prompt =f"""
        QUESTION:{user_message}
        INPUT_DOCUMENT:{user_document}
        EMBED_DATA: {search_output}
        EXTENDED_PAPER_INFO: {extended_info}
    

"""

contents = []
contents.append(types.Content(role="user", parts=[types.Part(text = prompt)]))
response_final = client.models.generate_content(
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction=instruction
        #tools=[orchestration_tools]
    ),
    contents = contents
)
print(response_final.text)

 - CALL: create_embedding()
 - CALL: search_embedded_documents(n = 5)
 - CALL: retrieve_documents(IDS = [89, 90, 91, 92])
The following papers are related to your input document based on the provided context:

1.  **Intelligent location of simultaneously active acoustic emission sources: Part I** by Unknown authors: This paper describes an intelligent acoustic emission source locator comprising a sensor antenna and a general regression neural network for solving location problems based on learning from examples.
2.  **General System theory, Like-Quantum Semantics and Fuzzy Sets** by Unknown authors: This paper explores extending quantum formalism within the general systems theory by using quantum semantics and considering the truth-values of quantum propositions within the context of fuzzy sets.
3.  **Real Options for Project Schedules (ROPS)** by Lester Ingber: This paper introduces Real Options for Project Schedules (ROPS), which uses recursive sampling/optimization shells to optimiz

In [153]:
#tool_call = response.candidates[0].content.parts[0].function_call

#if tool_call.name == "create_embedding":
#    result = create_embedding(**tool_call.args)

#function_response_part = types.Part.from_function_response(
#    name=tool_call.name,
#    response={"result": result},
#)

#contents.append(types.Content(role="model", parts=[types.Part(function_call=tool_call)])) # Append the model's function call message
#contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response
#response = client.models.generate_content(
#    model="gemini-2.0-flash", 
#    config=types.GenerateContentConfig(
#        system_instruction=instruction,
#        tools=[orchestration_tools]
#    ),
#    contents = contents
#)
#print(response)